In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import xarray as xr
import glob
import os
import re
import sys
import xarray as xr
import rioxarray as rio
dd_arpa = "c:\\Users\\m1865\\Desktop\\GIS Thesis Python\\ARPA"
sys.path.append(os.path.abspath("c:\\Users\\m1865\\Desktop\\GIS Thesis Python"))
cwd = os.getcwd()

In [ ]:
cams_NO2 = pd.read_csv(cwd + "\\CAMS NO2 Upscale.csv")
cams_O3 = pd.read_csv(cwd + "\\CAMS O3 Upscale.csv")
cams_PM10 = pd.read_csv(cwd + "\\CAMS PM10 Upscale.csv")
cams_PM25 = pd.read_csv(cwd + "\\CAMS PM25 Upscale.csv")

In [ ]:
cams_PM25

In [ ]:
arpa_NO2 = pd.read_csv(cwd + "\\ARPA NO2.csv").round({'EST_UTM':-3,'NORD_UTM':-3})
arpa_O3 = pd.read_csv(cwd + "\\ARPA O3.csv").round({'EST_UTM':-3,'NORD_UTM':-3})
arpa_PM10 = pd.read_csv(cwd + "\\ARPA PM10.csv").round({'EST_UTM':-3,'NORD_UTM':-3})
arpa_PM25 = pd.read_csv(cwd + "\\ARPA PM25.csv").round({'EST_UTM':-3,'NORD_UTM':-3})

In [ ]:
arpa_NO2

In [ ]:
arpa_NO2_sort = arpa_NO2.groupby(['NORD_UTM','EST_UTM','Month'],as_index=False).mean()
arpa_NO2_sort = arpa_NO2_sort.drop(columns=['ID_CELLA','Cella_X','Cella_Y'])
arpa_NO2_sort

In [ ]:
arpa_O3_sort = arpa_O3.groupby(['NORD_UTM','EST_UTM','Month'],as_index=False).mean()
arpa_O3_sort = arpa_O3_sort.drop(columns=['ID_CELLA','Cella_X','Cella_Y'])
arpa_PM10_sort = arpa_PM10.groupby(['NORD_UTM','EST_UTM','Month'],as_index=False).mean()
arpa_PM10_sort = arpa_PM10_sort.drop(columns=['ID_CELLA','Cella_X','Cella_Y'])
arpa_PM25_sort = arpa_PM25.groupby(['NORD_UTM','EST_UTM','Month'],as_index=False).mean()
arpa_PM25_sort = arpa_PM25_sort.drop(columns=['ID_CELLA','Cella_X','Cella_Y'])

In [ ]:
NO2 = pd.merge(arpa_NO2_sort,cams_NO2,on=['NORD_UTM','EST_UTM'])
NO2

In [ ]:
O3 = pd.merge(arpa_O3_sort,cams_O3,on=['NORD_UTM','EST_UTM'])
PM10 = pd.merge(arpa_PM10_sort,cams_PM10,on=['NORD_UTM','EST_UTM'])
PM25 = pd.merge(arpa_PM25_sort,cams_PM25,on=['NORD_UTM','EST_UTM'])

In [ ]:
NO2_bymonth = []
for i in range(12):
    NO2_bymonth.append(NO2[(NO2['Month_x']==(i+1))&(NO2['Month_y']==(i+1))])

In [ ]:
NO2_bymonth[0]

In [ ]:
O3_bymonth = []
PM10_bymonth = []
PM25_bymonth = []
for i in range(12):
    O3_bymonth.append(O3[(O3['Month_x']==(i+1))&(O3['Month_y']==(i+1))])
    PM10_bymonth.append(PM10[(PM10['Month_x']==(i+1))&(PM10['Month_y']==(i+1))])
    PM25_bymonth.append(PM25[(PM25['Month_x']==(i+1))&(PM25['Month_y']==(i+1))])

In [ ]:
from scipy.stats import pearsonr

NO2_Pearson = pd.DataFrame(columns=['Pollutant','Month','Pearson Coefficient','p value'])
for i in range(12):
    r_pearson, p_pearson= pearsonr(NO2_bymonth[i]['Valore'],NO2_bymonth[i][['no2_conc']])
    r_pearson = r_pearson[0]
    r_pearson = float(f"{r_pearson:.6f}")
    p_pearson = float(f"{p_pearson:.6f}")
    df = pd.DataFrame({
        'Pollutant': ['NO2'],
        'Month': [i+1],
        'Pearson Coefficient': [r_pearson],
        'p value': [p_pearson]
    })
    NO2_Pearson = pd.concat([NO2_Pearson,df])

In [ ]:
NO2_Pearson

In [ ]:
O3_Pearson = pd.DataFrame(columns=['Pollutant','Month','Pearson Coefficient','p value'])
PM10_Pearson = pd.DataFrame(columns=['Pollutant','Month','Pearson Coefficient','p value'])
PM25_Pearson = pd.DataFrame(columns=['Pollutant','Month','Pearson Coefficient','p value'])
for i in range(12):
    r_pearson, p_pearson= pearsonr(O3_bymonth[i]['Valore'],O3_bymonth[i][['o3_conc']])
    r_pearson = r_pearson[0]
    r_pearson = float(f"{r_pearson:.6f}")
    p_pearson = float(f"{p_pearson:.6f}")
    df = pd.DataFrame({
        'Pollutant': ['O3'],
        'Month': [i+1],
        'Pearson Coefficient': [r_pearson],
        'p value': [p_pearson]
    })
    O3_Pearson = pd.concat([O3_Pearson,df])
for i in range(12):
    r_pearson, p_pearson= pearsonr(PM10_bymonth[i]['Valore'],PM10_bymonth[i][['pm10_conc']])
    r_pearson = r_pearson[0]
    r_pearson = float(f"{r_pearson:.6f}")
    p_pearson = float(f"{p_pearson:.6f}")
    df = pd.DataFrame({
        'Pollutant': ['PM10'],
        'Month': [i+1],
        'Pearson Coefficient': [r_pearson],
        'p value': [p_pearson]
    })
    PM10_Pearson = pd.concat([PM10_Pearson,df])
for i in range(12):
    r_pearson, p_pearson= pearsonr(PM25_bymonth[i]['Valore'],PM25_bymonth[i][['pm2p5_conc']])
    r_pearson = r_pearson[0]
    r_pearson = float(f"{r_pearson:.6f}")
    p_pearson = float(f"{p_pearson:.6f}")
    df = pd.DataFrame({
        'Pollutant': ['PM25'],
        'Month': [i+1],
        'Pearson Coefficient': [r_pearson],
        'p value': [p_pearson]
    })
    PM25_Pearson = pd.concat([PM25_Pearson,df])

In [ ]:
O3_Pearson

In [ ]:
PM10_Pearson

In [ ]:
PM25_Pearson

In [ ]:
Pearson_ALL = [NO2_Pearson,O3_Pearson,PM10_Pearson,PM25_Pearson]
Pearson_ALL = pd.concat(Pearson_ALL)
Pearson_ALL.to_excel('Pearson Coefficient of ARPA and CAMS.xlsx', index=False)